In [55]:
%matplotlib inline
import pandas as pd
import os 
import numpy as np
import xarray as xr
from scipy.spatial import cKDTree

In [52]:
direc = '/Users/diana/Dropbox/UW/Research/rasm/25_km'
filename = 'vic_initdata_wr50a.08311989'
initdata_file = os.path.join(direc, filename)
initdata = pd.read_csv(initdata_file, 
                       delim_whitespace=True, 
                       index_col=None, 
                       skiprows=1, 
                       names=['shum', 'snow', 'rad1', 'rad2'])

In [54]:
domain_25km_filename = 'domain.lnd.wr25b_ar9v4.170413.nc'
direc = '/Users/diana/Dropbox/UW/Research/rasm/25_km'
domain_25km = xr.open_dataset(os.path.join(direc, domain_25km_filename))

domain_50km_filename = 'domain.lnd.wr50a_ar9v4.100920.nc'
domain_50km = xr.open_dataset(os.path.join(direc, domain_50km_filename))

In [59]:
def lon_lat_to_cartesian(lon, lat, R = 1):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius R
    """
    lon_r = np.radians(lon)
    lat_r = np.radians(lat)

    x = R * np.cos(lat_r) * np.cos(lon_r)
    y = R * np.cos(lat_r) * np.sin(lon_r)
    z = R * np.sin(lat_r)

    return x, y, z

In [70]:
lon_50km = domain_50km['xc'].values.flatten()
lat_50km = domain_50km['yc'].values.flatten()
lon_25km = domain_25km['xc'].values.flatten()
lat_25km = domain_25km['yc'].values.flatten()

In [77]:
lon_25km.shape

(227563,)

In [71]:
# lon, lat from coarser res data
xs, ys, zs = lon_lat_to_cartesian(lon_50km, lat_50km) 
# lon, lat from higher res data
xt, yt, zt = lon_lat_to_cartesian(lon_25km, lat_25km)

In [72]:
# build KD tree
zipped_50km = np.dstack(([xs, ys, zs]))[0]
tree = cKDTree(zipped_50km)

In [73]:
# find indices of the nearest neighbors in the flattened array 
zipped_25km = np.dstack(([xt, yt, zt]))[0]
d, inds = tree.query(zipped_25km, k=1)

In [95]:
shum_nearest = initdata['shum'].values.flatten()[inds]
snow_nearest = initdata['snow'].values.flatten()[inds]
rad1_nearest = initdata['rad1'].values.flatten()[inds]
rad2_nearest = initdata['rad2'].values.flatten()[inds]

In [96]:
initdata_25km = initdata.copy(deep=True)

In [97]:
initdata_25km_reindexed = initdata_25km.reindex(np.arange(0, len(lon_25km)))
initdata_25km_reindexed['shum'] = shum_nearest
initdata_25km_reindexed['snow'] = snow_nearest
initdata_25km_reindexed['rad1'] = rad1_nearest
initdata_25km_reindexed['rad2'] = rad2_nearest

try writing to text file

In [102]:
initdata_25km_reindexed.to_csv('test_write.txt', 
                               sep=' ', 
                               index=True,
                               mode='w',
                               header=False, 
                               encoding='utf-8', 
                               float_format='%1.4f')